In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [5]:
endpoint = r"https://www.sec.gov/cgi-bin/browse-edgar"


param_dict = {'action':'getcompany',
              'CIK':'1265107',
              'type':'10-k',
              'dateb':'20190101',
              'owner':'exclude',
              'start':'',
              'output':'',
              'count':'100'}


response = requests.get(url = endpoint, params = param_dict)
soup = BeautifulSoup(response.content, 'html.parser')


print('Request Successful')
print(response.url)

Request Successful
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1265107&type=10-k&dateb=20190101&owner=exclude&start=&output=&count=100


In [6]:

doc_table = soup.find_all('table', class_='tableFile2')


base_url_sec = r"https://www.sec.gov"

master_list = []


for row in doc_table[0].find_all('tr')[0:3]:
    
    
    cols = row.find_all('td')
    
    
    if len(cols) != 0:        
        
        
        filing_type = cols[0].text.strip()                 
        filing_date = cols[3].text.strip()
        filing_numb = cols[4].text.strip()
        
        
        filing_doc_href = cols[1].find('a', {'href':True, 'id':'documentsbutton'})       
        filing_int_href = cols[1].find('a', {'href':True, 'id':'interactiveDataBtn'})
        filing_num_href = cols[4].find('a')
        
        
        if filing_doc_href != None:
            filing_doc_link = base_url_sec + filing_doc_href['href'] 
        else:
            filing_doc_link = 'no link'
        
        
        if filing_int_href != None:
            filing_int_link = base_url_sec + filing_int_href['href'] 
        else:
            filing_int_link = 'no link'
        
        
        if filing_num_href != None:
            filing_num_link = base_url_sec + filing_num_href['href'] 
        else:
            filing_num_link = 'no link'
        
        
        file_dict = {}
        file_dict['file_type'] = filing_type
        file_dict['file_number'] = filing_numb
        file_dict['file_date'] = filing_date
        file_dict['links'] = {}
        file_dict['links']['documents'] = filing_doc_link
        file_dict['links']['interactive_data'] = filing_int_link
        file_dict['links']['filing_number'] = filing_num_link
    
        
        print('-'*100)        
        print("Filing Type: " + filing_type)
        print("Filing Date: " + filing_date)
        print("Filing Number: " + filing_numb)
        print("Document Link: " + filing_doc_link)
        print("Filing Number Link: " + filing_num_link)
        print("Interactive Data Link: " + filing_int_link)
        
        
        master_list.append(file_dict)


----------------------------------------------------------------------------------------------------
Filing Type: 10-K
Filing Date: 2018-03-07
Filing Number: 333-11002518671437
Document Link: https://www.sec.gov/Archives/edgar/data/1265107/000126510718000013/0001265107-18-000013-index.htm
Filing Number Link: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-110025&owner=exclude&count=100
Interactive Data Link: https://www.sec.gov/cgi-bin/viewer?action=view&cik=1265107&accession_number=0001265107-18-000013&xbrl_type=v
----------------------------------------------------------------------------------------------------
Filing Type: 10-K
Filing Date: 2017-03-13
Filing Number: 333-11002517683575
Document Link: https://www.sec.gov/Archives/edgar/data/1265107/000126510717000007/0001265107-17-000007-index.htm
Filing Number Link: https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-110025&owner=exclude&count=100
Interactive Data Link: https://www.sec.gov/c

In [7]:
for report in master_list[0:2]:
    
    print('-'*100)
    print(report['links']['documents'])
    print(report['links']['filing_number'])
    print(report['links']['interactive_data'])

----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/1265107/000126510718000013/0001265107-18-000013-index.htm
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-110025&owner=exclude&count=100
https://www.sec.gov/cgi-bin/viewer?action=view&cik=1265107&accession_number=0001265107-18-000013&xbrl_type=v
----------------------------------------------------------------------------------------------------
https://www.sec.gov/Archives/edgar/data/1265107/000126510717000007/0001265107-17-000007-index.htm
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-110025&owner=exclude&count=100
https://www.sec.gov/cgi-bin/viewer?action=view&cik=1265107&accession_number=0001265107-17-000007&xbrl_type=v


In [8]:

endpoint = r"https://www.sec.gov/cgi-bin/browse-edgar"


param_dict = {'action':'getcompany',
              'CIK':'1265107',
              'type':'10-k',
              'dateb':'20190101',
              'owner':'exclude',
              'start':'',
              'output':'atom',
              'count':'100'}


response = requests.get(url = endpoint, params = param_dict)
soup = BeautifulSoup(response.content, 'lxml')


print('Request Successful')
print(response.url)

Request Successful
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1265107&type=10-k&dateb=20190101&owner=exclude&start=&output=atom&count=100


In [10]:
entries = soup.find_all('entry')


master_list_xml = []


for entry in entries[0:2]:
    

    accession_num = entry.find('accession-number').text
    

    entry_dict = {}
    entry_dict[accession_num] = {}
    
   
    category_info = entry.find('category')    
    entry_dict[accession_num]['category'] = {}
    entry_dict[accession_num]['category']['label'] = category_info['label']
    entry_dict[accession_num]['category']['scheme'] = category_info['scheme']
    entry_dict[accession_num]['category']['term'] =  category_info['term']


    entry_dict[accession_num]['file_info'] = {}
    entry_dict[accession_num]['file_info']['act'] = entry.find('act').text
    entry_dict[accession_num]['file_info']['file_number'] = entry.find('file-number').text
    entry_dict[accession_num]['file_info']['file_number_href'] = entry.find('file-number-href').text
    entry_dict[accession_num]['file_info']['filing_date'] =  entry.find('filing-date').text
    entry_dict[accession_num]['file_info']['filing_href'] = entry.find('filing-href').text
    entry_dict[accession_num]['file_info']['filing_type'] =  entry.find('filing-type').text
    entry_dict[accession_num]['file_info']['form_number'] =  entry.find('film-number').text
    entry_dict[accession_num]['file_info']['form_name'] =  entry.find('form-name').text
    entry_dict[accession_num]['file_info']['file_size'] =  entry.find('size').text
    
   
    entry_dict[accession_num]['request_info'] = {}
    entry_dict[accession_num]['request_info']['link'] =  entry.find('link')['href']
    entry_dict[accession_num]['request_info']['title'] =  entry.find('title').text
    entry_dict[accession_num]['request_info']['last_updated'] =  entry.find('updated').text
    
    
    master_list_xml.append(entry_dict)
    
    print('-'*100)
    print(entry.find('form-name').text)
    print(entry.find('file-number').text)
    print(entry.find('file-number-href').text)
    print(entry.find('link')['href'])

----------------------------------------------------------------------------------------------------
Annual report [Section 13 and 15(d), not S-K Item 405]
333-110025
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-110025&owner=exclude&count=100
https://www.sec.gov/Archives/edgar/data/1265107/000126510718000013/0001265107-18-000013-index.htm
----------------------------------------------------------------------------------------------------
Annual report [Section 13 and 15(d), not S-K Item 405]
333-110025
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=333-110025&owner=exclude&count=100
https://www.sec.gov/Archives/edgar/data/1265107/000126510717000007/0001265107-17-000007-index.htm


In [11]:
import pprint
pprint.pprint(master_list_xml[0]['0001265107-18-000013']['category'])

{'label': 'form type', 'scheme': 'https://www.sec.gov/', 'term': '10-K'}


In [12]:

endpoint = r"https://www.sec.gov/cgi-bin/browse-edgar"


param_dict = {'action':'getcompany',
              'CIK':'1265107',
              'dateb':'20190101',
              'owner':'exclude',
              'start':'',
              'output':'atom',
              'count':'100'}


response = requests.get(url = endpoint, params = param_dict)
soup = BeautifulSoup(response.content, 'lxml')


links = soup.find_all('link',{'rel':'next'})


while soup.find_all('link',{'rel':'next'}) != []:

    
    next_page_link = links[0]['href']  
    
    print('-'*100)
    print(next_page_link)
    

    response = requests.get(url = next_page_link)
    soup = BeautifulSoup(response.content, 'lxml')
    
    
    links = soup.find_all('link',{'rel':'next'})

----------------------------------------------------------------------------------------------------
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001265107&type=&datea=&dateb=20190101&owner=exclude&count=100&output=atom&start=100
